In [1]:
import pip
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

In [2]:
# Import necessary libraries
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint

In [3]:
# Checking all the ones in NYSE and NASDAQ
f = open('./foe.txt','r')
file_list = f.readlines()
FOE = ['^GSPC']
for line in file_list: 
    splited_line =line.split(':') 
    if splited_line[0] in ['NYSE', 'NASDAQ']:
        FOE.append(splited_line[1].strip())
print(FOE)

['^GSPC', 'MMM', 'ADBE', 'AMZN', 'ADSK', 'SAM', 'KMX', 'CMG', 'CB', 'CTSH', 'CL', 'COST', 'DIS', 'FDX', 'JWN', 'PNRA', 'QCOM', 'SLB', 'LUV', 'SBUX', 'TROW', 'UPS', 'WFM']


In [4]:
# Set start to end dates
start_date = '1999-01-01'
end_date = '2019-01-01'

In [5]:
appended_data = []

for ticker in FOE: 
    print(ticker)
#     left PNRA and wFM due to no logner traded 
    if ticker == 'PNRA' or  ticker == 'WFM':
        continue
    stock_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
    appended_data.append(stock_data['Close'])

result = pd.concat(appended_data, axis=1)    
# pprint(result)

^GSPC
MMM
ADBE
AMZN
ADSK
SAM
KMX
CMG
CB
CTSH
CL
COST
DIS
FDX
JWN
PNRA
QCOM
SLB
LUV
SBUX
TROW
UPS
WFM


In [6]:
# Rename Column names
FOE.remove('PNRA')
FOE.remove('WFM')
print(FOE)
result.columns = FOE

['^GSPC', 'MMM', 'ADBE', 'AMZN', 'ADSK', 'SAM', 'KMX', 'CMG', 'CB', 'CTSH', 'CL', 'COST', 'DIS', 'FDX', 'JWN', 'QCOM', 'SLB', 'LUV', 'SBUX', 'TROW', 'UPS']


In [7]:
# Write Daily data
result.to_csv('./FOE_YH.csv')
# Write Normalized Daily data
result_norm = (result - result.mean()) / (result.max() - result.min())
result_norm.to_csv('./FOE_norm_YH.csv')

In [8]:
# Resample data to Monthly
df = pd.read_csv("./FOE_YH.csv", parse_dates =["Date"], index_col ="Date") 
Monthly_resampled_data = df.resample('M').mean()

In [9]:
# Write the Monthly data
Monthly_resampled_data.to_csv('./FOE_Monthly.csv')

In [10]:
# Write normalized Monthly data
Monthly_resampled_data_norm = (Monthly_resampled_data - Monthly_resampled_data.mean()) / (Monthly_resampled_data.max() - Monthly_resampled_data.min())
Monthly_resampled_data_norm.to_csv('./FOE_norm_Monthly.csv')

In [11]:
# Function to get the first None Empty Value
def get_first_values(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [12]:
# Get percentage change from Monthly data
First_non_NaN = Monthly_resampled_data.apply(get_first_values)
percent_change_monthly = Monthly_resampled_data/ First_non_NaN
pprint(percent_change_monthly)
percent_change_monthly.to_csv('./FOE_percent_change_Monthly_1999_base.csv')

               ^GSPC       MMM       ADBE       AMZN       ADSK        SAM  \
Date                                                                         
1999-01-31  1.000000  1.000000   1.000000   1.000000   1.000000   1.000000   
1999-02-28  0.998244  1.017967   0.903289   0.814403   0.933515   0.943153   
1999-03-31  1.026337  0.996546   1.020444   1.002902   0.916065   0.889882   
1999-04-30  1.068853  1.041824   1.210800   1.357082   0.654633   0.861416   
1999-05-31  1.066705  1.209430   1.486144   0.996295   0.596536   0.871541   
...              ...       ...        ...        ...        ...        ...   
2018-09-30  2.323478  5.635067  43.634646  29.309551  13.559320  33.224741   
2018-10-31  2.230558  5.345169  41.303822  26.566255  12.353327  30.362101   
2018-11-30  2.180723  5.340193  39.082631  24.232102  12.090797  32.382467   
2018-12-31  2.055861  5.177278  38.000162  23.247593  11.734494  28.236654   
2019-01-31  2.009994  5.090828  36.807808  22.944772  11.533086 

In [13]:
percent_change_monthly_mtm = percent_change_monthly.pct_change()
percent_change_monthly_mtm.to_csv('./FOE_mtm_change.csv')